# Demographics with ADHD. First load in ADHD df

In [161]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency,chisquare
import numpy as np

In [162]:
#filtered_data_arms1 = pd.read_csv("/home/faird/lundq163/projects/FRF_results_viz/GBM_work/list_ADHDscores_ARMS2_arms1xgb_merged.csv")
#filtered_data_arms2= pd.read_csv("/home/faird/lundq163/projects/FRF_results_viz/GBM_work/list_ADHDscores_ARMS1_arms2xgb_merged.csv")
data_arms1 = pd.read_csv("/home/feczk001/shared/projects/FEZ_USERS/feczk001/UPPS_ABCD_FRF/code/jacob/ADHDscores_list_ARMS1_merged.csv")
data_arms2 = pd.read_csv("/home/feczk001/shared/projects/FEZ_USERS/feczk001/UPPS_ABCD_FRF/code/jacob/ADHDscores_list_ARMS2_merged.csv")
data_arms1

,subjectkey,ADHD1,ADHD2,ADHD3,ADHD4,ADHDcomposite,community
0,NDAR_INV003RTV85,Ctrl,Ctrl,Ctrl,Ctrl,-0.638587,7
1,NDAR_INV007W6H7B,Ctrl,Ctrl,Ctrl,Ctrl,-0.189392,7
2,NDAR_INV00BD7VDC,Ctrl,Ctrl,Ctrl,Ctrl,0.907719,5
3,NDAR_INV00HEV6HB,Ctrl,Ctrl,Ctrl,Ctrl,-0.254566,1
4,NDAR_INV00LH735Y,Ctrl,Ctrl,Ctrl,Ctrl,-0.526288,8
...,...,...,...,...,...,...,...
3332,NDAR_INVZY8L7CT1,Ctrl,Ctrl,Ctrl,Ctrl,-0.077094,4
3333,NDAR_INVZYC44GB8,Ctrl,Ctrl,Ctrl,Ctrl,0.251977,2
3334,NDAR_INVZZ05KY5J,Ctrl,Ctrl,Ctrl,Ctrl,0.017155,1
3335,NDAR_INVZZFG6J5U,Ctrl,Ctrl,Ctrl,Ctrl,0.002618,4


In [163]:
#currently keeping all communities, regardless of size, because we are trying to evaluate across the arms with the overfit xgb model

communities_more_than_100_arms1 = data_arms1['community'].value_counts()[data_arms1['community'].value_counts() > 100].index
communities_more_than_100_arms2 = data_arms2['community'].value_counts()[data_arms2['community'].value_counts() > 100].index
filtered_data_arms1 = data_arms1[data_arms1['community'].isin(communities_more_than_100_arms1)]
filtered_data_arms2 = data_arms2[data_arms2['community'].isin(communities_more_than_100_arms2)]
data_arms1['community'].value_counts()

community
2     661
1     631
4     449
3     415
5     327
6     267
7     184
8     108
9     104
10     89
11     66
12     35
13      1
Name: count, dtype: int64

In [164]:
#for xgboost communities, add 1
#filtered_data_arms1['community'] = filtered_data_arms1['community'] + 1
#filtered_data_arms2['community'] = filtered_data_arms2['community'] + 1

In [165]:
#convert community to a categorical variable to ensure proper ordering on the x-axis
filtered_data_arms1.loc[:, 'community'] = pd.Categorical(filtered_data_arms1['community'])
filtered_data_arms2.loc[:, 'community'] = pd.Categorical(filtered_data_arms2['community'])

In [166]:
filtered_data_arms1['community'].value_counts()

community
2    661
1    631
4    449
3    415
5    327
6    267
7    184
8    108
9    104
Name: count, dtype: int64

In [167]:
#merge in latest version of participants.tsv
participant_demographics = pd.read_csv("/home/midb-ig/shared/projects/ABCD/Automated_QC/brain_coverage/participants.tsv", sep='\t')
participant_demographics = participant_demographics.rename(columns={'participant_id':'subjectkey'})
participant_demographics['subjectkey'] = participant_demographics['subjectkey'].str.replace('sub-NDAR', 'NDAR_')
participant_demographics_first_ses = participant_demographics.drop_duplicates(subset='subjectkey', keep='first')
participant_demographics_first_ses

,subjectkey,session_id,collection_3165,site,scanner_manufacturer,scanner_model,scanner_software,matched_group,sex,White,...,bc_rest_run-1,bc_rest_run-2,bc_nback_run-1,bc_nback_run-2,#subcortical_segmentation_vol_out (n=22),#cortical_morphometry_sulc_out (n=333),5min_pconn,10min_pconn,#pconn_out_5min (n=61776),#pconn_out_10min (n=61776)
0,NDAR_INV003RTV85,ses-baselineYear1Arm1,1,site06,SIEMENS,Prisma_fit,syngo MR E11,1,2,1,...,Pass,Pass,Pass,Pass,0.0,3.0,Pass,Pass,89.0,75.0
1,NDAR_INV005V6D2C,ses-baselineYear1Arm1,0,site10,GE MEDICAL SYSTEMS,DISCOVERY MR750,"[27, LX, MR Software release:DV26.0_R01_1725.a]",2,2,1,...,Pass,Pass,Pass,Pass,1.0,7.0,Fail,Fail,NaN,NaN
2,NDAR_INV007W6H7B,ses-baselineYear1Arm1,1,site22,GE MEDICAL SYSTEMS,DISCOVERY MR750,"[25, LX, MR Software release:DV25.0_R02_1549.b]",1,1,1,...,Pass,Pass,Pass,Pass,3.0,0.0,Pass,Pass,125.0,125.0
3,NDAR_INV00BD7VDC,ses-baselineYear1Arm1,1,site07,SIEMENS,Prisma_fit,syngo MR E11,1,1,1,...,Pass,Pass,Pass,Pass,0.0,1.0,Pass,Pass,111.0,101.0
4,NDAR_INV00CY2MDM,ses-2YearFollowUpYArm1,1,site20,SIEMENS,Prisma,syngo MR E11,1,1,1,...,Pass,Pass,Pass,Pass,0.0,3.0,Pass,Pass,115.0,231.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22942,NDAR_INVZZPKBDAC,ses-baselineYear1Arm1,0,site12,SIEMENS,Prisma_fit,syngo MR E11,1,2,1,...,Pass,Pass,Pass,Pass,0.0,0.0,Pass,Pass,310.0,399.0
22944,NDAR_INVZZZ2ALR6,ses-6YearFollowUpYArm1,0,888,888,888,888,2,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,Fail,Fail,NaN,NaN
22947,NDAR_INVZZZNB0XC,ses-baselineYear1Arm1,1,site03,SIEMENS,Prisma,syngo MR E11,2,2,1,...,Pass,Fail,Pass,Fail,1.0,0.0,Pass,Pass,19.0,22.0
22949,NDAR_INVZZZP87KR,ses-2YearFollowUpYArm1,1,site19,Philips Medical Systems,Ingenia,"[5.3.1, 5.3.1.2]",2,2,0,...,NaN,NaN,NaN,NaN,1.0,NaN,Fail,Fail,NaN,NaN


In [168]:
data_arms1_demo_adhd = filtered_data_arms1.merge(participant_demographics_first_ses, on='subjectkey', how='left')
data_arms2_demo_adhd = filtered_data_arms2.merge(participant_demographics_first_ses, on='subjectkey', how='left')
#data_arms1comm_xgb_demo_adhd = arms2data_arms1comm.merge(participant_demographics_first_ses, on='subjectkey', how='left')
#data_arms2comm_xgb_demo_adhd = arms1data_arms2comm.merge(participant_demographics_first_ses, on='subjectkey', how='left')
data_arms1_demo_adhd.dtypes

subjectkey                                            object
ADHD1                                                 object
ADHD2                                                 object
ADHD3                                                 object
ADHD4                                                 object
ADHDcomposite                                        float64
community                                              int64
session_id                                            object
collection_3165                                        int64
site                                                  object
scanner_manufacturer                                  object
scanner_model                                         object
scanner_software                                      object
matched_group                                          int64
sex                                                    int64
White                                                  int64
Black/African American  

In [169]:
data_arms1_demo_adhd = data_arms1_demo_adhd.drop(['session_id','collection_3165','site','scanner_manufacturer','scanner_model','scanner_software','matched_group','bc_MID_run-2','bc_SST_run-1','bc_MID_run-1','bc_rest_run-1','bc_rest_run-2','bc_nback_run-1','bc_nback_run-2','#subcortical_segmentation_vol_out (n=22)','#cortical_morphometry_sulc_out (n=333)','5min_pconn','10min_pconn','#pconn_out_5min (n=61776)','#pconn_out_10min (n=61776)'], axis=1)
data_arms2_demo_adhd = data_arms2_demo_adhd.drop(['session_id','collection_3165','site','scanner_manufacturer','scanner_model','scanner_software','matched_group','bc_MID_run-2','bc_SST_run-1','bc_MID_run-1','bc_rest_run-1','bc_rest_run-2','bc_nback_run-1','bc_nback_run-2','#subcortical_segmentation_vol_out (n=22)','#cortical_morphometry_sulc_out (n=333)','5min_pconn','10min_pconn','#pconn_out_5min (n=61776)','#pconn_out_10min (n=61776)'], axis=1)
#data_arms1comm_xgb_demo_adhd = data_arms1comm_xgb_demo_adhd.drop(['session_id','collection_3165','site','scanner_manufacturer','scanner_model','scanner_software','matched_group','bc_MID_run-2','bc_SST_run-1','bc_MID_run-1','bc_rest_run-1','bc_rest_run-2','bc_nback_run-1','bc_nback_run-2','#subcortical_segmentation_vol_out (n=22)','#cortical_morphometry_sulc_out (n=333)','5min_pconn','10min_pconn','#pconn_out_5min (n=61776)','#pconn_out_10min (n=61776)'], axis=1)
#data_arms2comm_xgb_demo_adhd = data_arms2comm_xgb_demo_adhd.drop(['session_id','collection_3165','site','scanner_manufacturer','scanner_model','scanner_software','matched_group','bc_MID_run-2','bc_SST_run-1','bc_MID_run-1','bc_rest_run-1','bc_rest_run-2','bc_nback_run-1','bc_nback_run-2','#subcortical_segmentation_vol_out (n=22)','#cortical_morphometry_sulc_out (n=333)','5min_pconn','10min_pconn','#pconn_out_5min (n=61776)','#pconn_out_10min (n=61776)'], axis=1)
data_arms1_demo_adhd

,subjectkey,ADHD1,ADHD2,ADHD3,ADHD4,ADHDcomposite,community,sex,White,Black/African American,...,age,handedness,siblings_twins,income,participant_education,parental_education,anesthesia_exposure,pc1,pc2,pc3
0,NDAR_INV003RTV85,Ctrl,Ctrl,Ctrl,Ctrl,-0.638587,7,2,1,0,...,131,1,0,8,5,13,0,0.278102,0.340003,0.237957
1,NDAR_INV007W6H7B,Ctrl,Ctrl,Ctrl,Ctrl,-0.189392,7,1,1,0,...,126,1,0,10,4,19,2,0.983973,0.098231,0.797205
2,NDAR_INV00BD7VDC,Ctrl,Ctrl,Ctrl,Ctrl,0.907719,5,1,1,0,...,112,1,0,10,4,20,1,0.150013,0.839135,0.862830
3,NDAR_INV00HEV6HB,Ctrl,Ctrl,Ctrl,Ctrl,-0.254566,1,1,0,1,...,149,888,0,999,7,888,0,0.050440,-0.000863,-0.406094
4,NDAR_INV00LH735Y,Ctrl,Ctrl,Ctrl,Ctrl,-0.526288,8,1,1,0,...,109,1,0,6,3,13,0,-1.035351,-1.182294,0.436951
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3141,NDAR_INVZY8L7CT1,Ctrl,Ctrl,Ctrl,Ctrl,-0.077094,4,1,1,0,...,130,2,0,7,5,21,1,0.493073,-0.409751,-0.746371
3142,NDAR_INVZYC44GB8,Ctrl,Ctrl,Ctrl,Ctrl,0.251977,2,1,1,0,...,131,1,888,10,5,21,1,0.910144,0.814503,-1.084629
3143,NDAR_INVZZ05KY5J,Ctrl,Ctrl,Ctrl,Ctrl,0.017155,1,1,0,1,...,122,2,1,4,4,13,0,-0.920924,-0.326607,-0.000983
3144,NDAR_INVZZFG6J5U,Ctrl,Ctrl,Ctrl,Ctrl,0.002618,4,2,1,0,...,129,1,0,7,4,19,2,-0.313337,0.371288,-0.997517


In [170]:
data_arms1_race_adhd = data_arms1_demo_adhd.drop(['sex','age','handedness','siblings_twins','income','participant_education','parental_education','anesthesia_exposure','pc1','pc2','pc3'], axis=1)
data_arms2_race_adhd = data_arms2_demo_adhd.drop(['sex','age','handedness','siblings_twins','income','participant_education','parental_education','anesthesia_exposure','pc1','pc2','pc3'], axis=1)
#data_arms1comm_xgb_race_adhd = data_arms1comm_xgb_demo_adhd.drop(['sex','age','handedness','siblings_twins','income','participant_education','parental_education','anesthesia_exposure','pc1','pc2','pc3'], axis=1)
#data_arms2comm_xgb_race_adhd = data_arms2comm_xgb_demo_adhd.drop(['sex','age','handedness','siblings_twins','income','participant_education','parental_education','anesthesia_exposure','pc1','pc2','pc3'], axis=1)
#to be used at bottom
data_arms1_race_adhd.dtypes

subjectkey                                            object
ADHD1                                                 object
ADHD2                                                 object
ADHD3                                                 object
ADHD4                                                 object
ADHDcomposite                                        float64
community                                              int64
White                                                  int64
Black/African American                                 int64
American Indian, Native American                       int64
Alaska Native                                          int64
Native Hawaiian                                        int64
Guamanian                                              int64
Samoan                                                 int64
Other Pacific Islander                                 int64
Asian Indian                                           int64
Chinese                 

In [171]:
data_arms1_income_adhd = data_arms1_demo_adhd.drop(['ADHDcomposite','sex','age','handedness','siblings_twins','participant_education','parental_education','anesthesia_exposure','pc1','pc2','pc3','White','Black/African American','American Indian, Native American','Alaska Native','Native Hawaiian','Guamanian','Samoan','Other Pacific Islander','Asian Indian','Chinese','Filipino','Japanese','Korean','Vietnamese','Other Asian','Other Race','Refuse to Answer',"Don't Know",'Do you consider the child Hispanic/Latino/Latina?'], axis=1)
data_arms2_income_adhd = data_arms2_demo_adhd.drop(['ADHDcomposite','sex','age','handedness','siblings_twins','participant_education','parental_education','anesthesia_exposure','pc1','pc2','pc3','White','Black/African American','American Indian, Native American','Alaska Native','Native Hawaiian','Guamanian','Samoan','Other Pacific Islander','Asian Indian','Chinese','Filipino','Japanese','Korean','Vietnamese','Other Asian','Other Race','Refuse to Answer',"Don't Know",'Do you consider the child Hispanic/Latino/Latina?'], axis=1)
#data_arms1comm_xgb_income_adhd = data_arms1comm_xgb_demo_adhd.drop(['ADHDcomposite','sex','age','handedness','siblings_twins','participant_education','parental_education','anesthesia_exposure','pc1','pc2','pc3','White','Black/African American','American Indian, Native American','Alaska Native','Native Hawaiian','Guamanian','Samoan','Other Pacific Islander','Asian Indian','Chinese','Filipino','Japanese','Korean','Vietnamese','Other Asian','Other Race','Refuse to Answer',"Don't Know",'Do you consider the child Hispanic/Latino/Latina?'], axis=1)
#data_arms2comm_xgb_income_adhd = data_arms2comm_xgb_demo_adhd.drop(['ADHDcomposite','sex','age','handedness','siblings_twins','participant_education','parental_education','anesthesia_exposure','pc1','pc2','pc3','White','Black/African American','American Indian, Native American','Alaska Native','Native Hawaiian','Guamanian','Samoan','Other Pacific Islander','Asian Indian','Chinese','Filipino','Japanese','Korean','Vietnamese','Other Asian','Other Race','Refuse to Answer',"Don't Know",'Do you consider the child Hispanic/Latino/Latina?'], axis=1)
#to be used at bottom
data_arms1_income_adhd

,subjectkey,ADHD1,ADHD2,ADHD3,ADHD4,community,income
0,NDAR_INV003RTV85,Ctrl,Ctrl,Ctrl,Ctrl,7,8
1,NDAR_INV007W6H7B,Ctrl,Ctrl,Ctrl,Ctrl,7,10
2,NDAR_INV00BD7VDC,Ctrl,Ctrl,Ctrl,Ctrl,5,10
3,NDAR_INV00HEV6HB,Ctrl,Ctrl,Ctrl,Ctrl,1,999
4,NDAR_INV00LH735Y,Ctrl,Ctrl,Ctrl,Ctrl,8,6
...,...,...,...,...,...,...,...
3141,NDAR_INVZY8L7CT1,Ctrl,Ctrl,Ctrl,Ctrl,4,7
3142,NDAR_INVZYC44GB8,Ctrl,Ctrl,Ctrl,Ctrl,2,10
3143,NDAR_INVZZ05KY5J,Ctrl,Ctrl,Ctrl,Ctrl,1,4
3144,NDAR_INVZZFG6J5U,Ctrl,Ctrl,Ctrl,Ctrl,4,7


In [172]:
def add_adhd_column(df):
    adhd_columns = [col for col in df.columns if col.startswith('ADHD')]

    df['ADHD'] = df[adhd_columns].apply(
        lambda row: any(str(val).startswith('ADHD') for val in row),
        axis=1
    )

    return df

In [173]:
data_arms1_income_adhd_bin = add_adhd_column(data_arms1_income_adhd)
data_arms2_income_adhd_bin = add_adhd_column(data_arms2_income_adhd)
#data_arms1comm_xgb_income_adhd_bin = add_adhd_column(data_arms1comm_xgb_income_adhd)
#data_arms2comm_xgb_income_adhd_bin = add_adhd_column(data_arms2comm_xgb_income_adhd)
data_arms1_income_adhd_bin = data_arms1_income_adhd_bin.drop(['ADHD1','ADHD2','ADHD3','ADHD4'], axis=1)
data_arms2_income_adhd_bin = data_arms2_income_adhd_bin.drop(['ADHD1','ADHD2','ADHD3','ADHD4'], axis=1)
#data_arms1comm_xgb_income_adhd_bin = data_arms1comm_xgb_income_adhd_bin.drop(['ADHD1','ADHD2','ADHD3','ADHD4'], axis=1)
#data_arms2comm_xgb_income_adhd_bin = data_arms2comm_xgb_income_adhd_bin.drop(['ADHD1','ADHD2','ADHD3','ADHD4'], axis=1)

data_arms1_income_adhd_bin

,subjectkey,community,income,ADHD
0,NDAR_INV003RTV85,7,8,False
1,NDAR_INV007W6H7B,7,10,False
2,NDAR_INV00BD7VDC,5,10,False
3,NDAR_INV00HEV6HB,1,999,False
4,NDAR_INV00LH735Y,8,6,False
...,...,...,...,...
3141,NDAR_INVZY8L7CT1,4,7,False
3142,NDAR_INVZYC44GB8,2,10,False
3143,NDAR_INVZZ05KY5J,1,4,False
3144,NDAR_INVZZFG6J5U,4,7,False


In [174]:
def group_income(df, income_groups):
    df_copy = df.copy()
    df_copy['income_grouped'] = np.nan
    for group_name, group_values in income_groups.items():
        group_values = [str(val) for val in group_values]
        df_copy.loc[df_copy['income'].astype(str).isin(group_values), 'income_grouped'] = group_name
    df_grouped = df_copy.dropna(subset=['income_grouped'])
    return df_grouped

income_groups = {
    'poverty': ['1', '2', '3', '4'],
    'low': ['5', '6'],
    'middle': ['7', '8'],
    'high': ['9'],
    'extravagent': ['10']
}

In [175]:
arms1_grouped_income_adhd = group_income(data_arms1_income_adhd_bin, income_groups).drop(['income'], axis=1)
arms1_grouped_income_adhd

/tmp/ipykernel_292595/3016154451.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'poverty' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_copy.loc[df_copy['income'].astype(str).isin(group_values), 'income_grouped'] = group_name


,subjectkey,community,ADHD,income_grouped
0,NDAR_INV003RTV85,7,False,middle
1,NDAR_INV007W6H7B,7,False,extravagent
2,NDAR_INV00BD7VDC,5,False,extravagent
4,NDAR_INV00LH735Y,8,False,low
5,NDAR_INV00R4TXET,2,False,high
...,...,...,...,...
3141,NDAR_INVZY8L7CT1,4,False,middle
3142,NDAR_INVZYC44GB8,2,False,extravagent
3143,NDAR_INVZZ05KY5J,1,False,poverty
3144,NDAR_INVZZFG6J5U,4,False,middle


In [176]:
arms1_grouped_income_adhd['community'].value_counts()

community
2    563
1    536
4    384
3    351
5    287
6    212
7    150
9     88
8     86
Name: count, dtype: int64

In [177]:
def count_adhd_subjects_by_community(df):
    adhd_counts = df.groupby(['community','income_grouped'])['ADHD'].value_counts().unstack(fill_value=0)
    adhd_counts.columns = ['ADHD_false', 'ADHD_true']
    #adhd_counts['Total_Count'] = adhd_counts['False_Count'] + adhd_counts['True_Count']
    #adhd_counts['True_Percentage'] = (adhd_counts['True_Count'] / adhd_counts['Total_Count'] * 100).round(2)
    
    return adhd_counts

In [178]:
arms1_income_adhd_count = count_adhd_subjects_by_community(arms1_grouped_income_adhd)
arms1_income_adhd_count

ADHD_false  ADHD_true
community income_grouped                       
1         extravagent             61          4
          high                   183         13
          low                     64          4
          middle                 133         12
          poverty                 58          4
2         extravagent             99          4
          high                   208         10
          low                     47          1
          middle                 155          7
          poverty                 32          0
3         extravagent             35          2
          high                   118          9
          low                     40          0
          middle                  96          2
          poverty                 45          4
4         extravagent             30          3
          high                    74         15
          low                     68          5
          middle                  87         20
          poverty                 72         10
5         extravagent             66          3
          high                   110          6
          low                     23          1
          middle                  58          5
          poverty                 13          2
6         extravagent             21          1
          high                    56          6
          low                     21          3
          middle                  59          5
          poverty                 36          4
7         extravagent             24          2
          high                    42          5
          low                     16          1
          middle                  36          5
          poverty                 15          4
8         extravagent             15          1
          high                    29          0
          low                      9          1
          middle                  27          2
          poverty                  2          0
9         extravagent             12          1
          high                    25          2
          low                      8          0
          middle                  28          1
          poverty                 10          1

In [145]:
arms1_income_adhd_count.to_csv('./arms1_frf_adhd_count_comm_income.csv')

# organizing race/ethnicity information

In [179]:
data_arms1_race_adhd

,subjectkey,ADHD1,ADHD2,ADHD3,ADHD4,ADHDcomposite,community,White,Black/African American,"American Indian, Native American",...,Chinese,Filipino,Japanese,Korean,Vietnamese,Other Asian,Other Race,Refuse to Answer,Don't Know,Do you consider the child Hispanic/Latino/Latina?
0,NDAR_INV003RTV85,Ctrl,Ctrl,Ctrl,Ctrl,-0.638587,7,1,0,0,...,0,0,0,0,0,0,0,0,0,2
1,NDAR_INV007W6H7B,Ctrl,Ctrl,Ctrl,Ctrl,-0.189392,7,1,0,0,...,0,0,0,0,0,0,0,0,0,2
2,NDAR_INV00BD7VDC,Ctrl,Ctrl,Ctrl,Ctrl,0.907719,5,1,0,0,...,0,0,0,0,0,0,0,0,0,2
3,NDAR_INV00HEV6HB,Ctrl,Ctrl,Ctrl,Ctrl,-0.254566,1,0,1,0,...,0,0,0,0,0,0,0,0,0,2
4,NDAR_INV00LH735Y,Ctrl,Ctrl,Ctrl,Ctrl,-0.526288,8,1,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3141,NDAR_INVZY8L7CT1,Ctrl,Ctrl,Ctrl,Ctrl,-0.077094,4,1,0,0,...,0,0,0,0,0,0,0,0,0,2
3142,NDAR_INVZYC44GB8,Ctrl,Ctrl,Ctrl,Ctrl,0.251977,2,1,0,0,...,0,0,0,0,0,0,0,0,0,2
3143,NDAR_INVZZ05KY5J,Ctrl,Ctrl,Ctrl,Ctrl,0.017155,1,0,1,0,...,0,0,0,0,0,0,0,0,0,2
3144,NDAR_INVZZFG6J5U,Ctrl,Ctrl,Ctrl,Ctrl,0.002618,4,1,0,0,...,0,0,0,0,0,0,0,0,0,2


In [180]:
data_arms1_race_adhd.dtypes

subjectkey                                            object
ADHD1                                                 object
ADHD2                                                 object
ADHD3                                                 object
ADHD4                                                 object
ADHDcomposite                                        float64
community                                              int64
White                                                  int64
Black/African American                                 int64
American Indian, Native American                       int64
Alaska Native                                          int64
Native Hawaiian                                        int64
Guamanian                                              int64
Samoan                                                 int64
Other Pacific Islander                                 int64
Asian Indian                                           int64
Chinese                 

In [181]:
data_arms1_race_adhd_bin = add_adhd_column(data_arms1_race_adhd)
data_arms2_race_adhd_bin = add_adhd_column(data_arms2_race_adhd)
#data_arms1comm_xgb_race_adhd_bin = add_adhd_column(data_arms1comm_xgb_race_adhd)
#data_arms2comm_xgb_race_adhd_bin = add_adhd_column(data_arms2comm_xgb_race_adhd)
data_arms1_race_adhd_bin = data_arms1_race_adhd_bin.drop(['ADHD1','ADHD2','ADHD3','ADHD4','ADHDcomposite','Do you consider the child Hispanic/Latino/Latina?','Refuse to Answer',"Don't Know"], axis=1)
data_arms2_race_adhd_bin = data_arms2_race_adhd_bin.drop(['ADHD1','ADHD2','ADHD3','ADHD4','ADHDcomposite','Do you consider the child Hispanic/Latino/Latina?','Refuse to Answer',"Don't Know"], axis=1)
#data_arms1comm_xgb_race_adhd_bin = data_arms1comm_xgb_race_adhd_bin.drop(['ADHD1','ADHD2','ADHD3','ADHD4'], axis=1)
#data_arms2comm_xgb_race_adhd_bin = data_arms2comm_xgb_race_adhd_bin.drop(['ADHD1','ADHD2','ADHD3','ADHD4'], axis=1)

data_arms1_race_adhd_bin.dtypes

subjectkey                          object
community                            int64
White                                int64
Black/African American               int64
American Indian, Native American     int64
Alaska Native                        int64
Native Hawaiian                      int64
Guamanian                            int64
Samoan                               int64
Other Pacific Islander               int64
Asian Indian                         int64
Chinese                              int64
Filipino                             int64
Japanese                             int64
Korean                               int64
Vietnamese                           int64
Other Asian                          int64
Other Race                           int64
ADHD                                  bool
dtype: object

In [182]:
def group_race(df, race_columns, race_groups):
    df_copy = df.copy()
    df_copy['race_grouped'] = np.nan
    
    for group_name, group_values in race_groups.items():
        group_mask = df_copy[race_columns].eq(1).any(axis=1) & \
                     df_copy[race_columns].loc[:, group_values].eq(1).any(axis=1)
        
        df_copy.loc[group_mask, 'race_grouped'] = group_name
    
    return df_copy

race_groups = {
    'White': ['White'],
    'Black/African American': ['Black/African American'],
    'American Indian, Native American, Alaska Native': ['American Indian, Native American', 'Alaska Native'],
    'Pacific Islander': ['Native Hawaiian', 'Guamanian', 'Samoan', 'Other Pacific Islander'],
    'Asian': ['Asian Indian', 'Chinese', 'Filipino', 'Japanese', 'Korean', 'Vietnamese', 'Other Asian'],
    'Other Race': ['Other Race'],
    #'Unspecified': ['Refuse to Answer', 'Don\'t Know']
}

In [183]:
race_columns = [
    'White', 'Black/African American', 'American Indian, Native American', 
    'Alaska Native', 'Native Hawaiian', 'Guamanian', 'Samoan', 
    'Other Pacific Islander', 'Asian Indian', 'Chinese', 'Filipino', 'Japanese', 
    'Korean', 'Vietnamese', 'Other Asian', 'Other Race'
]

arms1_grouped_race_adhd = group_race(data_arms1_race_adhd_bin, race_columns, race_groups)
arms1_grouped_race_adhd['race_grouped'].value_counts(dropna=False)

/tmp/ipykernel_292595/1689229312.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'White' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_copy.loc[group_mask, 'race_grouped'] = group_name


race_grouped
White                                              2172
Black/African American                              500
Asian                                               177
Other Race                                          169
American Indian, Native American, Alaska Native      82
NaN                                                  31
Pacific Islander                                     15
Name: count, dtype: int64

In [184]:
arms1_grouped_race_adhd.dtypes

subjectkey                          object
community                            int64
White                                int64
Black/African American               int64
American Indian, Native American     int64
Alaska Native                        int64
Native Hawaiian                      int64
Guamanian                            int64
Samoan                               int64
Other Pacific Islander               int64
Asian Indian                         int64
Chinese                              int64
Filipino                             int64
Japanese                             int64
Korean                               int64
Vietnamese                           int64
Other Asian                          int64
Other Race                           int64
ADHD                                  bool
race_grouped                        object
dtype: object

In [185]:
arms1_grouped_race_adhd_cleaned = arms1_grouped_race_adhd.drop(['White', 'Black/African American', 'American Indian, Native American', 'Alaska Native', 'Native Hawaiian', 'Guamanian', 'Samoan', 'Other Pacific Islander', 'Asian Indian', 'Chinese', 'Filipino', 'Japanese', 'Korean', 'Vietnamese', 'Other Asian', 'Other Race'], axis=1)
arms1_grouped_race_adhd_cleaned

,subjectkey,community,ADHD,race_grouped
0,NDAR_INV003RTV85,7,False,White
1,NDAR_INV007W6H7B,7,False,White
2,NDAR_INV00BD7VDC,5,False,White
3,NDAR_INV00HEV6HB,1,False,Black/African American
4,NDAR_INV00LH735Y,8,False,White
...,...,...,...,...
3141,NDAR_INVZY8L7CT1,4,False,White
3142,NDAR_INVZYC44GB8,2,False,White
3143,NDAR_INVZZ05KY5J,1,False,Black/African American
3144,NDAR_INVZZFG6J5U,4,False,White


In [186]:
arms1_grouped_race_adhd_cleaned['community'].value_counts()

community
2    661
1    631
4    449
3    415
5    327
6    267
7    184
8    108
9    104
Name: count, dtype: int64

In [187]:
def count_adhd_subjects_by_community_race(df):
    adhd_counts = df.groupby(['community','race_grouped'])['ADHD'].value_counts().unstack(fill_value=0)
    adhd_counts.columns = ['ADHD_false', 'ADHD_true']
    #adhd_counts['Total_Count'] = adhd_counts['False_Count'] + adhd_counts['True_Count']
    #adhd_counts['True_Percentage'] = (adhd_counts['True_Count'] / adhd_counts['Total_Count'] * 100).round(2)
    
    return adhd_counts

In [188]:
arms1_race_adhd_count = count_adhd_subjects_by_community_race(arms1_grouped_race_adhd_cleaned)
arms1_race_adhd_count

ADHD_false  \
community race_grouped                                                  
1         American Indian, Native American, Alaska Native          14   
          Asian                                                    36   
          Black/African American                                   99   
          Other Race                                               43   
          Pacific Islander                                          5   
          White                                                   387   
2         American Indian, Native American, Alaska Native          14   
          Asian                                                    49   
          Black/African American                                   50   
          Other Race                                               29   
          Pacific Islander                                          1   
          White                                                   487   
3         American Indian, Native American, Alaska Native           9   
          Asian                                                    10   
          Black/African American                                   77   
          Other Race                                               16   
          White                                                   279   
4         American Indian, Native American, Alaska Native          15   
          Asian                                                    20   
          Black/African American                                  100   
          Other Race                                               29   
          Pacific Islander                                          3   
          White                                                   218   
5         American Indian, Native American, Alaska Native           5   
          Asian                                                    23   
          Black/African American                                   24   
          Other Race                                               12   
          Pacific Islander                                          2   
          White                                                   242   
6         American Indian, Native American, Alaska Native           9   
          Asian                                                    12   
          Black/African American                                   53   
          Other Race                                               14   
          Pacific Islander                                          2   
          White                                                   150   
7         American Indian, Native American, Alaska Native           7   
          Asian                                                     8   
          Black/African American                                   25   
          Other Race                                                7   
          White                                                   116   
8         American Indian, Native American, Alaska Native           3   
          Asian                                                     1   
          Black/African American                                    8   
          Other Race                                                5   
          Pacific Islander                                          2   
          White                                                    83   
9         Asian                                                     9   
          Black/African American                                   17   
          Other Race                                                6   
          White                                                    65   

                                                           ADHD_true  
community race_grouped                                                
1         American Indian, Native American, Alaska Native          3  
          Asian                                  

In [189]:
arms1_race_adhd_count.to_csv('./arms1_frf_adhd_count_comm_race.csv')

# stopping here for now

In [176]:
def count_adhd_subjects_in_all(df):
    df.drop(['community'], axis=1)
    adhd_counts = df.groupby(['income_grouped'])['ADHD'].value_counts().unstack(fill_value=0)
    adhd_counts.columns = ['ADHD_false', 'ADHD_true']
    #adhd_counts['Total_Count'] = adhd_counts['False_Count'] + adhd_counts['True_Count']
    #adhd_counts['True_Percentage'] = (adhd_counts['True_Count'] / adhd_counts['Total_Count'] * 100).round(2)
    
    return adhd_counts

In [177]:
arms1_income_adhd_count_all = count_adhd_subjects_in_all(arms1_grouped_income_adhd)
arms1_income_adhd_count_all

,ADHD_false,ADHD_true
income_grouped,,
extravagent,363,21
high,845,66
low,296,16
middle,679,59
poverty,283,29


In [178]:
arms1_income_adhd_count_all.sum()

ADHD_false    2466
ADHD_true      191
dtype: int64

In [179]:
arms1_income_adhd_count_div = arms1_income_adhd_count_all.divide(2657)
arms1_income_adhd_count_div

,ADHD_false,ADHD_true
income_grouped,,
extravagent,0.136620,0.007904
high,0.318028,0.024840
low,0.111404,0.006022
middle,0.255551,0.022205
poverty,0.106511,0.010915


In [180]:
arms1_income_adhd_count.index.get_level_values('community').unique()

Index([1, 2, 3, 4, 5, 6, 7, 8, 9], dtype='int64', name='community')

In [181]:
arms1_income_adhd_count.loc[4]

,ADHD_false,ADHD_true
income_grouped,,
extravagent,30,3
high,74,15
low,68,5
middle,87,20
poverty,72,10


In [182]:
arms1_income_adhd_count_comm4_exp = arms1_income_adhd_count_div * 384
arms1_income_adhd_count_comm4_exp

,ADHD_false,ADHD_true
income_grouped,,
extravagent,52.462175,3.035002
high,122.122695,9.538577
low,42.779074,2.312382
middle,98.131728,8.526910
poverty,40.900263,4.191193


In [188]:
arms1_income_adhd_count_comm4_exp_rounded = arms1_income_adhd_count_comm4_exp.round(0)
arms1_income_adhd_count_comm4_exp_rounded

,ADHD_false,ADHD_true
income_grouped,,
extravagent,52.0,3.0
high,122.0,10.0
low,43.0,2.0
middle,98.0,9.0
poverty,41.0,4.0


In [189]:
chi2_income, p_value_income = chisquare(arms1_income_adhd_count.loc[4], arms1_income_adhd_count_comm4_exp_rounded)  
print(f"Observed values: {arms1_income_adhd_count.loc[4].tolist()}")
print(f"Expected values: {arms1_income_adhd_count_comm4_exp_rounded.tolist()}")
print(f"Chi-square statistic: {chi2_income:.5g}")
print(f"p-value: {p_value_income:.5g}")
print()

ValueError: For each axis slice, the sum of the observed frequencies must agree with the sum of the expected frequencies to a relative tolerance of 1e-08, but the percent differences are:
[0.0755287  0.89285714]

In [134]:
# Perform chi-square test of homogeneity
communities = arms1_income_adhd_count.index.get_level_values('community').unique()
chi_square_results = {}
for community in communities:
    community_df = arms1_income_adhd_count.loc[community]
    observed = community_df[['ADHD_false', 'ADHD_true']]
    chi2_income, p_value_income = chisquare(observed, expected)  

    chi_square_results[community] = {
        'chi2_statistic': chi2_income,
        'p_value': p_value_income,
        'degrees_of_freedom': dof_income,
        'observed_frequencies': contingency_table,
        'expected_frequencies': pd.DataFrame(
            expected_income, 
            index=contingency_table.index, 
            columns=contingency_table.columns
        )
    }
    
    print(f"\n--- Community {community} Results ---")
    print(f"Chi-square statistic: {chi2_income:.5g}")
    print(f"p-value: {p_value_income:.5g}")
    print(f"Degrees of freedom: {dof_income}")
    print("\nContingency Table (Observed Frequencies):")
    print(contingency_table)
    print("\nExpected Frequencies:")
    print(pd.DataFrame(
        expected_income, 
        index=contingency_table.index, 
        columns=contingency_table.columns
    ))

In [135]:
run_chi_square_by_community(arms1_income_adhd_count)


#Interpreting the results:

#If p-value < 0.05, reject null hypothesis: there is evidence that sex_at_birth differs across subgroups
#If p-value ≥ 0.05, fail to reject the null hypothesis: there is no evidence sex_at_birth differs across subgroups

#You might also want to look at the standardized residuals to see which specific cells contribute most to any significant difference:


--- Community 1 Results ---
Chi-square statistic: 0.63419
p-value: 0.95919
Degrees of freedom: 4

Contingency Table (Observed Frequencies):
                ADHD_false  ADHD_true
income_grouped                       
extravagent             61          4
high                   183         13
low                     64          4
middle                 133         12
poverty                 58          4

Expected Frequencies:
                ADHD_false  ADHD_true
income_grouped                       
extravagent      60.513060   4.486940
high            182.470149  13.529851
low              63.305970   4.694030
middle          134.990672  10.009328
poverty          57.720149   4.279851

--- Community 2 Results ---
Chi-square statistic: 2.0687
p-value: 0.72313
Degrees of freedom: 4

Contingency Table (Observed Frequencies):
                ADHD_false  ADHD_true
income_grouped                       
extravagent             99          4
high                   208         10
low         

{1: {'chi2_statistic': 0.6341946549257943,
  'p_value': 0.9591891310973556,
  'degrees_of_freedom': 4,
  'observed_frequencies':                 ADHD_false  ADHD_true
  income_grouped                       
  extravagent             61          4
  high                   183         13
  low                     64          4
  middle                 133         12
  poverty                 58          4,
  'expected_frequencies':                 ADHD_false  ADHD_true
  income_grouped                       
  extravagent      60.513060   4.486940
  high            182.470149  13.529851
  low              63.305970   4.694030
  middle          134.990672  10.009328
  poverty          57.720149   4.279851},
 2: {'chi2_statistic': 2.0686764497847876,
  'p_value': 0.7231289956638263,
  'degrees_of_freedom': 4,
  'observed_frequencies':                 ADHD_false  ADHD_true
  income_grouped                       
  extravagent             99          4
  high                   208         10

# setting your counts dataframe of interest here

In [22]:
df_pivot_counts = proportion_data_arms1.pivot(index='community', columns='ADHD_label', values='count')
df_pivot_counts = df_pivot_counts.sort_index()
df_pivot_counts

ADHD_label,ADHD1,ADHD2,ADHD3,ADHD4,Ctrl
community,,,,,
1,0,16,8,16,591
2,1,11,7,5,637
3,2,8,5,5,395
4,6,19,8,27,389
5,0,9,4,4,310
6,3,11,3,6,244
7,2,5,3,10,164
8,0,0,3,2,103
9,0,4,1,1,98


In [23]:
df_pivot_counts_no_ctrl = df_pivot_counts.drop(columns='Ctrl')
df_pivot_counts_no_ctrl

ADHD_label,ADHD1,ADHD2,ADHD3,ADHD4
community,,,,
1,0,16,8,16
2,1,11,7,5
3,2,8,5,5
4,6,19,8,27
5,0,9,4,4
6,3,11,3,6
7,2,5,3,10
8,0,0,3,2
9,0,4,1,1


In [24]:
adhd_columns = ['ADHD1', 'ADHD2', 'ADHD3', 'ADHD4']
df_adhd_total = df_pivot_counts.copy()
df_adhd_total['ADHD_total'] = df_adhd_total[adhd_columns].sum(axis=1)
df_adhd_total

ADHD_label,ADHD1,ADHD2,ADHD3,ADHD4,Ctrl,ADHD_total
community,,,,,,
1,0,16,8,16,591,40
2,1,11,7,5,637,24
3,2,8,5,5,395,20
4,6,19,8,27,389,60
5,0,9,4,4,310,17
6,3,11,3,6,244,23
7,2,5,3,10,164,20
8,0,0,3,2,103,5
9,0,4,1,1,98,6


In [25]:
df_adhd_totals = df_adhd_total.drop(columns=adhd_columns)
df_adhd_totals

ADHD_label,Ctrl,ADHD_total
community,,
1,591,40
2,637,24
3,395,20
4,389,60
5,310,17
6,244,23
7,164,20
8,103,5
9,98,6
